<a href="https://colab.research.google.com/github/MalekBhz/GoWithMe/blob/main/objets_trouv%C3%A9s_groupe_mysh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark py4j

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("objets-trouvés").getOrCreate()

spark

In [ ]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=objets-trouvés>

In [ ]:
from pyspark.shell import spark

objetstrouves  = spark\
  .read\
  .option("inferSchema", "true")\
  .option("delimiter", ";")\
  .option("header", "true")\
  .csv("/content/objets-trouves-restitution.csv")\

objetstrouves.show()


+-------------------+----------------------------+--------------------+--------+--------------------+--------------------+---------------------+
|               Date|Date et heure de restitution|                Gare|Code UIC|     Nature d'objets|       Type d'objets|Type d'enregistrement|
+-------------------+----------------------------+--------------------+--------+--------------------+--------------------+---------------------+
|2022-01-01 07:03:34|         2022-01-22 11:23:48|        Lille Europe|87223263|  Téléphone portable|Appareils électro...|         Objet trouvé|
|2022-01-01 10:23:21|         2022-01-02 10:37:01|           Perpignan|87784009|Sac d'enseigne (p...|Bagagerie: sacs, ...|         Objet trouvé|
|2022-01-01 10:55:22|         2022-01-27 10:49:05|        Lille Europe|87223263|           Sac à dos|Bagagerie: sacs, ...|         Objet trouvé|
|2022-01-01 11:26:54|         2022-01-03 13:14:27|                 Dax|87673202|           Sac à dos|Bagagerie: sacs, ...|        

# Nouvelle section

# Nouvelle section

#Objets les plus trouvés


In [ ]:
from pyspark.sql.functions import col

lostObjectsNature = objetstrouves.select("Nature d'objets")
countsObjectsNature = lostObjectsNature.groupby(
    col("Nature d'objets")).count()
countsObjectsNature.orderBy("count", ascending=False).show(truncate=100)

+------------------------------------------------------------------+-----+
|                                                   Nature d'objets|count|
+------------------------------------------------------------------+-----+
|                                                         Sac à dos|  390|
|                                       Porte-monnaie, portefeuille|  312|
|                                         Valise, sac sur roulettes|  284|
|                    Sac de voyage, sac de sport, sac à bandoulière|  183|
|                        Téléphone portable protégé (étui, coque,…)|  155|
|                                                        Sac à main|  152|
|                                                Téléphone portable|  127|
|                             Sac d'enseigne (plastique, papier, …)|   86|
|                   Carte d'identité, passeport, permis de conduire|   65|
|                      Manteau, veste, blazer, parka, blouson, cape|   49|
|Ordinateur, ordinateur p

#Top 5 villes qui trouvent le plus d'objets

In [ ]:
top5stations = objetstrouves.groupby(
    col("Gare")
    ).count().orderBy("count", ascending = False)
top5stations.show(5)

+-------------------+-----+
|               Gare|count|
+-------------------+-----+
| Paris Gare de Lyon|  196|
| Paris Montparnasse|  181|
| Paris Saint-Lazare|  138|
| Paris Gare du Nord|  113|
|Bordeaux Saint-Jean|   98|
+-------------------+-----+
only showing top 5 rows



# Types d'objets les plus trouvés

# Types d'objets trouvés par gare

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.functions import split

foundByStation = objetstrouves.groupby(
    col("Gare"), 
    col("Type d'objets")
    ).count().orderBy("count", ascending=False)
foundByStation = foundByStation.withColumn("ville",split(foundByStation["Gare"]," ").getItem(0))
foundByStation.show()


+--------------------+--------------------+-----+----------+
|                Gare|       Type d'objets|count|     ville|
+--------------------+--------------------+-----+----------+
|  Paris Montparnasse|Bagagerie: sacs, ...|   98|     Paris|
|  Paris Gare de Lyon|Bagagerie: sacs, ...|   94|     Paris|
|Marseille Saint-C...|Bagagerie: sacs, ...|   57| Marseille|
| Bordeaux Saint-Jean|Bagagerie: sacs, ...|   54|  Bordeaux|
|  Paris Gare du Nord|Bagagerie: sacs, ...|   53|     Paris|
|  Paris Saint-Lazare|Bagagerie: sacs, ...|   48|     Paris|
|  Paris Gare de Lyon|Appareils électro...|   45|     Paris|
|       Lyon Perrache|Bagagerie: sacs, ...|   42|      Lyon|
|      Lyon Part Dieu|Bagagerie: sacs, ...|   40|      Lyon|
|          Strasbourg|Bagagerie: sacs, ...|   37|Strasbourg|
|        Lille Europe|Bagagerie: sacs, ...|   37|     Lille|
|              Rennes|Bagagerie: sacs, ...|   36|    Rennes|
|              Nantes|Bagagerie: sacs, ...|   35|    Nantes|
|   Toulouse Matabiau|Ba

#Top 5 villes qui trouvent le plus d'objet

In [ ]:
foundByStation.createOrReplaceTempView("foundByStation") 

query=spark.sql("""SELECT ville,sum(count) as count FROM foundByStation group by ville order by count desc""")
query.show(5)


+---------+-----+
|    ville|count|
+---------+-----+
|    Paris|  691|
|     Lyon|  153|
| Bordeaux|   98|
|   Rennes|   87|
|Marseille|   86|
+---------+-----+
only showing top 5 rows



#Sur toutes les villes la liste de tous les objets trouvés

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.functions import split
from pyspark.sql.functions import col
from pyspark.sql.functions import col
foundByStation = objetstrouves.groupby(
    col("Gare"), 
    col("Type d'objets")
    ).count().orderBy("count", ascending=False)
foundByStation = foundByStation.withColumn("ville",split(foundByStation["Gare"]," ").getItem(0))

data = foundByStation.select(col("Type d'objets").alias('type_objet'),col("Gare"),col("count"),col("ville"))
# data.show()
data.createOrReplaceTempView("data") 
query=spark.sql("""SELECT ville, array_agg(type_objet),sum(count) FROM data group by ville""")
query.show()





+----------------+------------------------+----------+
|           ville|collect_list(type_objet)|sum(count)|
+----------------+------------------------+----------+
|       Granville|    [Bagagerie: sacs,...|         8|
| Bourg-en-Bresse|    [Appareils électr...|         1|
|              Le|    [Bagagerie: sacs,...|        26|
|            Nice|    [Bagagerie: sacs,...|        27|
|   Saint-Quentin|    [Bagagerie: sacs,...|         5|
|     Montpellier|    [Bagagerie: sacs,...|        53|
|         Avignon|    [Bagagerie: sacs,...|        15|
|   Saint-Nazaire|    [Bagagerie: sacs,...|        12|
|           Dijon|    [Bagagerie: sacs,...|        25|
|          Toulon|    [Bagagerie: sacs,...|         5|
|        Grenoble|    [Bagagerie: sacs,...|        13|
|        Chartres|    [Bagagerie: sacs,...|         6|
|       Trouville|    [Bagagerie: sacs,...|         1|
|        Boulogne|    [Bagagerie: sacs,...|         7|
|          Angers|    [Bagagerie: sacs,...|        12|
|       Du

#Objets Trouvés par mois

In [ ]:
from pyspark.sql.functions import month, mean, count

objectsFoundByMonth = (objetstrouves
                       .groupby(month("Date").alias("Month"))
                       .agg(count("Date").alias("count")))

objectsFoundByMonth.withColumnRenamed("count","objet_trouvé").show()


+-----+------------+
|Month|objet_trouvé|
+-----+------------+
|   12|           1|
|    1|        2336|
+-----+------------+

